# Socio-economic indicators evaluation

Read aggregated blocks spatial layer

In [1]:
import pandas as pd

blocks_gdf = pd.read_pickle('blocks.pickle').rename(columns={'territory_id':'parent'})

In [2]:
service_types_df = pd.read_pickle('service_types.pickle')

In [3]:
st_df = service_types_df[~service_types_df.blocksnet.isna()].copy()
st_df['service_type_id'] = st_df.index
agg_df = st_df.groupby('blocksnet').agg({'service_type_id': lambda s : list(s)})
new_columns = {}
for st_name, row in agg_df.iterrows():
    st_ids = row['service_type_id']
    for prefix in ['count', 'capacity']:
        sum_df = blocks_gdf[[f'{prefix}_{st_id}' for st_id in st_ids]].sum(axis=1)
        new_columns[f'{prefix}_{st_name}'] = sum_df
new_columns_df = pd.DataFrame.from_dict(new_columns)
blocks_gdf = pd.concat([blocks_gdf, new_columns_df], axis=1)

Read drive graph

In [4]:
from blocksnet.relations import get_accessibility_graph

graph = get_accessibility_graph(blocks_gdf, 'drive')

2025-10-16 16:09:16.573 | INFO     | iduedu.modules.drive_walk_builder:get_drive_graph_by_poly:91 - Downloading drive graph from OSM, it may take a while for large territory ...


Read drive accessibility matrix

In [5]:
acc_mx = pd.read_pickle('acc_mx.pickle')

## General indicators

In [6]:
from blocksnet.analysis.indicators.socio_economic import calculate_general_indicators

general_indicators = calculate_general_indicators(blocks_gdf)
general_indicators

,117,121,total
GENERAL | Area (km2),708.440170,1088.277523,1796.717693
GENERAL | Urbanization,0.005648,0.000000,0.002227


## Demographic indicators

In [7]:
from blocksnet.analysis.indicators.socio_economic import calculate_demographic_indicators

demographic_indicators = calculate_demographic_indicators(blocks_gdf.fillna(0).rename(columns={'territory_id':'parent'}))
demographic_indicators

,117,121,total
DEMOGRAPHIC | Population,14862.000000,279.000000,15141.000000
DEMOGRAPHIC | Density (people/km2),20.978483,0.256368,8.427033


## Transport indicators

In [8]:
from blocksnet.analysis.indicators.socio_economic import calculate_transport_indicators

transport_indicators = calculate_transport_indicators(blocks_gdf, acc_mx, graph)
transport_indicators

,117,121,total
TRANSPORT | Road network density (km/km2),0.270630,0.178112,0.214592
TRANSPORT | Settlements connectivity,0.258009,NaN,0.258009
TRANSPORT | Road network length (km),191.725428,193.835662,385.561090
TRANSPORT | Fuel stations count,5.000000,0.000000,5.000000
TRANSPORT | Average fuel station accessibility (min),36.193548,NaN,36.193548
TRANSPORT | Railway stops count,8.000000,4.000000,12.000000
TRANSPORT | Average railway stop accessibility (min),38.619355,NaN,38.619355
TRANSPORT | Airports count,1.000000,0.000000,1.000000
TRANSPORT | Average airport accessibility (min),24.412903,NaN,24.412903


## Engineering indicators

In [9]:
from blocksnet.analysis.indicators.socio_economic import calculate_engineering_indicators

engineering_indicators = calculate_engineering_indicators(blocks_gdf)
engineering_indicators

,117,121,total
ENGINEERING | Substation,0.0,0.0,0.0
ENGINEERING | Water works,0.0,0.0,0.0
ENGINEERING | Wastewater plant,0.0,0.0,0.0
ENGINEERING | Reservoir,NaN,NaN,NaN
ENGINEERING | Gas distribution,NaN,NaN,NaN
ENGINEERING | Infrastructure object,0.0,0.0,0.0


## Social indicators

In [10]:
from blocksnet.analysis.indicators.socio_economic import SocialIndicator

SOCIAL_INDICATORS_MAPPING = {
    SocialIndicator.EXTRACURRICULAR: [23, 24],
    SocialIndicator.AMBULANCE: [39, 40],
    SocialIndicator.SPECIAL_MEDICAL: [41],
    SocialIndicator.PREVENTIVE_MEDICAL: [42],
    SocialIndicator.GYM: [68],
    SocialIndicator.ORPHANAGE: [46],
    SocialIndicator.SOCIAL_SERVICE_CENTER: [43],
    SocialIndicator.CULTURAL_CENTER: [49],
    SocialIndicator.CONCERT_HALL: [53],
    SocialIndicator.ICE_ARENA: [60],
    SocialIndicator.ECO_TRAIL: [72],
    SocialIndicator.FIRE_STATION: [79],
    SocialIndicator.TOURIST_BASE: [112],
}

SERVICE_TYPES_IDS_MAPPING = {st_id:indicator for indicator, st_ids in SOCIAL_INDICATORS_MAPPING.items() for st_id in st_ids}

SERVICE_TYPES_IDS_MAPPING

{23: SOCIAL | Extracurricular,
 24: SOCIAL | Extracurricular,
 39: SOCIAL | Ambulance,
 40: SOCIAL | Ambulance,
 41: SOCIAL | Special medical,
 42: SOCIAL | Preventive medical,
 68: SOCIAL | Gym,
 46: SOCIAL | Orphanage,
 43: SOCIAL | Social service center,
 49: SOCIAL | Cultural center,
 53: SOCIAL | Concert hall,
 60: SOCIAL | Ice arena,
 72: SOCIAL | Eco trail,
 79: SOCIAL | Fire station,
 112: SOCIAL | Tourist base}

In [11]:
service_types_df['indicator'] = service_types_df.index.map(SERVICE_TYPES_IDS_MAPPING)
service_types_df = service_types_df.rename(columns={
    'radius_availability_meters': 'meters',
    'time_availability_minutes': 'minutes',
    'services_per_1000_normative': 'count',
    'services_capacity_per_1000_normative': 'capacity'
})

In [12]:
from blocksnet.analysis.indicators.socio_economic import calculate_social_indicators

sc_indicators, sp_indicators = calculate_social_indicators(blocks_gdf.fillna(0), acc_mx, acc_mx, service_types_df)

100%|██████████| 39/39 [00:01<00:00, 23.63it/s]


In [13]:
sc_indicators

,117,121,total
SOCIALCOUNT | Kindergarten,7,1,8
SOCIALCOUNT | School,4,1,5
SOCIALCOUNT | College,1,0,1
SOCIALCOUNT | University,0,0,0
SOCIALCOUNT | Extracurricular,3,0,3
SOCIALCOUNT | Hospital,1,0,1
SOCIALCOUNT | Polyclinic,1,0,1
SOCIALCOUNT | Ambulance,1,0,1
SOCIALCOUNT | Sanatorium,0,0,0
SOCIALCOUNT | Special medical,1,0,1


In [14]:
sp_indicators

,117,121,total
SOCIALPROVISION | Kindergarten,0.797210,0.000000,0.779643
SOCIALPROVISION | School,0.753872,0.054054,0.739837
SOCIALPROVISION | College,0.946667,0.000000,0.898734
SOCIALPROVISION | University,0.000000,0.000000,0.000000
SOCIALPROVISION | Extracurricular,NaN,NaN,NaN
SOCIALPROVISION | Hospital,0.012500,0.000000,0.011905
SOCIALPROVISION | Polyclinic,0.013393,0.000000,0.012876
SOCIALPROVISION | Ambulance,NaN,NaN,NaN
SOCIALPROVISION | Sanatorium,0.000000,0.000000,0.000000
SOCIALPROVISION | Special medical,NaN,NaN,NaN


## Concat results

In [15]:
pd.concat([
    general_indicators,
    demographic_indicators,
    transport_indicators,
    engineering_indicators,
    sc_indicators,
    sp_indicators,
])

,117,121,total
GENERAL | Area (km2),708.440170,1088.277523,1796.717693
GENERAL | Urbanization,0.005648,0.000000,0.002227
DEMOGRAPHIC | Population,14862.000000,279.000000,15141.000000
DEMOGRAPHIC | Density (people/km2),20.978483,0.256368,8.427033
TRANSPORT | Road network density (km/km2),0.270630,0.178112,0.214592
...,...,...,...
SOCIALPROVISION | Police,0.836538,0.000000,0.783784
SOCIALPROVISION | Hotel,0.950000,0.875000,0.946429
SOCIALPROVISION | Hostel,0.012500,0.000000,0.011905
SOCIALPROVISION | Tourist base,0.000000,0.000000,0.000000
